# 수행 프로세스
중간 산출물의 편의성을 위해 데이터 선택을 0단계로 정의함

task01: Step 0-2

task02: Step 3

- Step 0) 데이터 선택
- Step 1) 핵심 구문 추출(Key Phrase Extraction)&감정 분석(Sentiment Analysis)
  - 결과물: issue, sentiment, sentiments_all
- Step 2) 군집 분석(Clustering Analysis)
  - 결과물: theme, tid
- Step 3) 기회영역값 도출(Opportunity Score Extraction)
  - 결과물: issue별 긍정, 부정, 중립 개수

In [1]:
import pandas as pd

In [6]:
# task01의 결과인 S2_감성테마.pkl을 읽어들여 분석
result_s2 = pd.read_pickle('result/제공_네이버쇼핑리뷰_S2_감성테마.pkl')

In [7]:
result_s2

,tid,raw_text,issue,sentiment,sentiment_all,theme
0,text0001,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정,제품 품질 및 성능
1,text0002,너무 만족하며 사용중입니다,사용 만족,긍정,긍정,사용자 만족 및 추천
2,text0003,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정,설치 및 배송 서비스
3,text0003,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정,설치 및 배송 서비스
4,text0004,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정,사용자 만족 및 추천
...,...,...,...,...,...,...
2476,text0991,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,중립,긍정,캠핑 및 야외 활용
2477,text0991,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정,기대 및 만족도
2478,text0992,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정,화질 및 화면
2479,text0992,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정,사용 편의성 및 이동성


In [8]:
# 'issue'별 'sentiment' 개수 계산 및 'total' 컬럼 추가 후 내림차순 정렬
result_s3 = result_s2.groupby(['issue', 'theme', 'sentiment']).size().unstack(fill_value=0)
result_s3['total'] = result_s3.sum(axis=1)
result_s3 = result_s3.sort_values(by='total', ascending=False)
result_s3

,sentiment,긍정,부정,중립,total
issue,theme,,,,
빠른 배송,기타,179,0,0,179
좋은 제품,제품 품질 및 성능,111,0,0,111
무거움,문제 및 불만,0,73,8,81
저렴한 가격,가격 및 가치,54,0,0,54
좋은 화질,제품 품질 및 성능,47,0,0,47
...,...,...,...,...,...
빠른 구매 필요성,가격 및 구매,1,0,0,1
빠른 답변,설치 및 배송,1,0,0,1
빠른 문의 답장,구매 및 배송 만족,1,0,0,1


In [9]:
# Check if 'issue' and 'theme' have a 1:1 correspondence
issue_theme_unique_pairs = result_s2[['issue', 'theme']].drop_duplicates()
issue_theme_duplicated = issue_theme_unique_pairs.duplicated(subset='issue', keep=False)

# If there are any duplicated 'issue', it means 'issue' and 'theme' do not have a 1:1 correspondence
if issue_theme_duplicated.any():
    print("issue와 theme는 1:1 대응이 되지 않습니다.")
else:
    print("issue와 theme는 1:1 대응이 됩니다.")

issue와 theme는 1:1 대응이 됩니다.


In [10]:
# 엑셀 파일로 저장
result_s3.to_excel('result/제공_네이버쇼핑리뷰_S3_기회영역값.xlsx')